In [ ]:
import numpy as np
import scipy.integrate as integrate
from functools import partial
import matplotlib.pyplot as plt
plt.style.use("bmh")
rng = np.random.default_rng(2022)

# Example Notebook for `PystoMS.synthetics` Module

The synthetics module of PystoMS shall allow for sampling of isotopic patterns and features over a variety of technical setups, e.g. mass spectrometry (MS), liquid chromatography coupled MS (LC-MS) and LC-MS setups with ion mobility spectrometry as additional separation technique (LC-IMS-MS).

## Isotopic Distributions in Mass Spectrometry

Peptide isotopic patterns can be modeled via an gaussian mixture distribution. The positions of the components are dependent on the charge and the mass of the peptide while the weights can be calculated by models such as the averagine-like model presented by Breen *et al.*.

In PystoMS a `scipy.stats.rv_continuous` subclass is implemented to allow sampling from isotopic distributions. 

In [ ]:
from pystoms.synthetics import IsotopicAveragineDistribution
iso = IsotopicAveragineDistribution()

To show that `IsotopicAveragineDistribution.pdf()` is a valid probability distribution, one could
estimate the integral like so:

In [ ]:
pdf = partial(iso.pdf,loc=301.2,mass=301.2,charge=1,sigma=0.05,num_peaks=6)
Integration = integrate.quad(pdf,290,320)
print(Integration)

To calculate the pdf along an axis, the parameters of the isotopic distribution must be passed to the pdf method. 
The parameter `mass` is only considered in calculation of the component weights, to shift the distribution from 0 to the monoisotopic peak use `loc`. Both `mass` and `loc` should store the mass and the mass to charge ratio of the monoisotopic peak, respectively.

In [ ]:
x = np.arange(3000,3100)/10
y = iso.pdf(x,loc=301.2,mass=301.2,charge=2,sigma=0.05,num_peaks=6)
plt.plot(x,y)
plt.show()

To draw random samples from the distribution, use the `.rvs()` method.

In [ ]:
samples = iso.rvs( loc=301.2, mass = 301.2,charge = 2,sigma=0.05,num_peaks=6,size=5000,random_state=rng)
plt.hist(samples,bins=100)
plt.show()

Both `.pdf()` and `.rvs()` internally call the overwritten methods `._pdf()` and `._rvs()`, respectively. The `._rvs()` method first samples a component a sample is generated by according to the component weights. Then the sample's deviation from its component's mean is drawn from a normal distribution N(0,σ).
To simulate (m/z,intensities) form the distribution use the `.rvs_to_intensities()` method. The method draws `size` samples from
`.rvs()` and bins theses samples based on `bin_width`. Each sample is considered a single molecule that contributes `signal_per_molecule` arbitrary units
to intensity of it's bin.

In [ ]:
bins_pos,intensities = iso.rvs_to_intensities( loc=301.2, mass = 301.2,charge = 2,sigma=0.01,num_peaks=6,size=10000,random_state=rng,bin_width=0.0001)
plt.plot(bins_pos,intensities,alpha=0.5,ls=":")
plt.scatter(bins_pos,intensities)
plt.show()

It is also possible to pass a `sigma` vector of different sigmas (with size corresponding to the number of modeled peaks `num_peaks`).

In [ ]:
y2 = iso.pdf(x,loc=301.2,mass=301.2,charge=2,sigma=np.array([0.1,0.2,0.01,0.01]),num_peaks=4)
plt.plot(x,y2)
plt.show()

In [ ]:
samples2 = iso.rvs(loc=301.2,mass=301.2,charge=2,sigma=np.array([0.1,0.2,0.01,0.01]),num_peaks=4,size=50000)
plt.hist(samples2,bins=100)
plt.show()

In [ ]:
bins_pos2,intensities2 = iso.rvs_to_intensities( loc=301.2, mass = 301.2,charge = 2,sigma=np.array([0.1,0.2,0.01,0.01]),num_peaks=4,size=50000,random_state=rng,bin_width=0.001)
plt.plot(bins_pos2,intensities2,alpha=0.5,ls=":")
plt.scatter(bins_pos2,intensities2)
plt.show()